# Парсер новостных текстов с сайтов РБК и Лента.ру

## Импорт библиотек и описание классов

In [250]:
# Импорт библиотек
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython import display
import requests
from bs4 import BeautifulSoup

import json

In [251]:
dateFrom = '2020-01-01'
dateTo = '2020-05-05'

### Парсер для Лента.ру

In [252]:
class lentaRu_parser:
    def __init__(self):
        pass
    
    
    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from=0&'\                       # Смещение
        + 'size=1000&'\                    # Кол-во статей
        + 'sort=2&'\                       # Сортировка по дате (2), по релевантности (1)
        + 'title_only=0&'\                 # Точная фраза в заголовке
        + 'domain=1&'\                     # ??
        + 'modified%2Cformat=yyyy-MM-dd&'\ # Формат даты
        + 'type=1&'\                       # Материалы. Все материалы (0). Новость (1)
        + 'bloc=4&'\                       # Рубрика. Экономика (4). Все рубрики (0)
        + 'modified%2Cfrom=2020-01-01&'\
        + 'modified%2Cto=2020-11-01&'\
        + 'query='                         # Поисковой запрос
        """
        hasType = int(param_dict['type']) != 0
        hasBloc = int(param_dict['bloc']) != 0

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from={}&'.format(param_dict['from'])\
        + 'size={}&'.format(param_dict['size'])\
        + 'sort={}&'.format(param_dict['sort'])\
        + 'title_only={}&'.format(param_dict['title_only'])\
        + 'domain={}&'.format(param_dict['domain'])\
        + 'modified%2Cformat=yyyy-MM-dd&'\
        + 'type={}&'.format(param_dict['type']) * hasType\
        + 'bloc={}&'.format(param_dict['bloc']) * hasBloc\
        + 'modified%2Cfrom={}&'.format(param_dict['dateFrom'])\
        + 'modified%2Cto={}&'.format(param_dict['dateTo'])\
        + 'query={}'.format(param_dict['query'])
        print(url)
        
        return url


    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['matches'])
        search_table['PostDate']=param_dict['dateFrom']
        
        return search_table

    
    def get_articles(self,
                     param_dict,
                     time_step = 1,
                     save_every = 5, 
                     save_excel = True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса 
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%Y-%m-%d')
        dateTo = datetime.strptime(param_copy['dateTo'], '%Y-%m-%d')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')
        
        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime('%Y-%m-%d')
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime('%Y-%m-%d')
            # print('Parsing articles from '\
            #       + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            
            out = pd.concat([out, self._get_search_table(param_copy)], ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime('%Y-%m-%d')
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("/tmp/checkpoint_table.xlsx")
                print('Checkpoint saved!')
                save_counter = 0
            
        if save_excel:
            out.to_excel("lenta_{}_{}.xlsx".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
        print('Finish')
        
        return out

In [253]:


URL = 'https://www.rbc.ru/rbcfreenews/'


def get_soup(url):
    r = requests.get(url).text
    return BeautifulSoup(r, 'lxml')


def save_json(data):
    with open('rbk_data.json', "w") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)


def load_rbk_news(num_of_news = 15):
    news_data = {}
    df_rbk = pd.DataFrame()

    soup = get_soup(URL)

    # Получаем все ссылки на новости
    news_links = soup.find('div', class_='main').find_all('a', {'class': ['main__big__link', 'main__feed__link']})

    # Для каждой ссылки получаем информацию и записываем в news_data
    for i in range(num_of_news):

        link = news_links[i].get('href').split('?')[0]
        name = link
        news_data[name] = {}
        soup = get_soup(link)
        print(soup)

        # Переходим на страницу для дальнейшенго парсинга
        article = soup.find('div', class_='article')
        try:
            category = article.find('a', class_='article__header__category')
        except:
            continue
            
        date = article.find('time', class_='article__header__date')
        date = None if date is None else date .get('datetime').replace('T', ' ').split('+')[0]

        title = article.find('div', class_='article__header__title')
        image = article.find('div', class_='article__main-image')
        article_paragraphs = article.find_all('p')
        article_text = ''
        for paragraph in article_paragraphs:
            article_text += paragraph.text

        # Заполняем полученными данными news_data
        news_data[name]['link'] = link
        news_data[name]['date'] = date
        news_data[name]['text'] = article_text.replace('\xa0', '').replace('\n', '').replace('\r', '')

        try:
            news_data[name]['title'] = title.text.replace('\n', '')
        except AttributeError:
            news_data[name]['title'] = 'Без заголовка'
        try:
            news_data[name]['category'] = category.text.replace('\n', '')
        except AttributeError:
            news_data[name]['category'] = 'Без категории'
        try:
            news_data[name]['image'] = image.find('img').get('src')
        except AttributeError:
            news_data[name]['image'] = 'Без обложки'

    save_json(news_data)




## Пример выгрузки данных


### РБК.ру

In [254]:
# load_rbk_news(num_of_news = 15)

### Лента.ру

* __query__ - поисковой запрос (ключевое слово)

* __offset__ - cмещение поисковой выдачи (от 0 до __size__)

* __size__ - количество статей. Ограничено время запроса, точного лимита нет. 1000 работает почти всегда 

* __sort__ - сортировка по дате: (2) - по убыванию, (3) - по возрастанию; по релевантности (1) 

* __title_only__ - точная фраза в заголовке (1)

* __domain__ - ? 

* __material__ - материалы: Все материалы (0). Новость (1). ["0", "1", "2", "3", "4", ...]

* __block__ - рубрика: Экономика (4). Все рубрики (0). ["0", "1", "2", "3", "4", ...]

* __dateFrom__ - с даты

* __dateTo__ - по дату

_Чтобы не специфировать параметр, оставляем поле пустым_

In [255]:

# Задаем тут параметры
use_parser = "LentaRu"

query = ''
offset = 0
size = 1000
sort = "3"
title_only = "0"
domain = "1"
material = "0"
bloc = "4"
# dateFrom = '2020-01-01'
# dateTo = "2020-01-05"

if use_parser == "LentaRu":
    param_dict = {'query'     : query, 
                  'from'      : str(offset),
                  'size'      : str(size),
                  'dateFrom'  : dateFrom,
                  'dateTo'    : dateTo,
                  'sort'      : sort,
                  'title_only': title_only,
                  'type'      : material, 
                  'bloc'      : bloc,
                  'domain'    : domain}

print(use_parser, "- param_dict:", param_dict)
lentaRu_parser()

LentaRu - param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2020-01-01', 'dateTo': '2020-05-05', 'sort': '3', 'title_only': '0', 'type': '0', 'bloc': '4', 'domain': '1'}


In [256]:
assert use_parser == "LentaRu"
parser = lentaRu_parser()
tbl = parser.get_articles(param_dict=param_dict,
                          time_step = 1,
                          save_every = 5, 
                          save_excel = True)
print(len(tbl.index))
tbl.head()

Checkpoint saved!
https://lenta.ru/search/v2/process?from=0&size=1000&sort=3&title_only=0&domain=1&modified%2Cformat=yyyy-MM-dd&bloc=4&modified%2Cfrom=2020-04-30&modified%2Cto=2020-05-01&query=
https://lenta.ru/search/v2/process?from=0&size=1000&sort=3&title_only=0&domain=1&modified%2Cformat=yyyy-MM-dd&bloc=4&modified%2Cfrom=2020-05-02&modified%2Cto=2020-05-03&query=
https://lenta.ru/search/v2/process?from=0&size=1000&sort=3&title_only=0&domain=1&modified%2Cformat=yyyy-MM-dd&bloc=4&modified%2Cfrom=2020-05-04&modified%2Cto=2020-05-05&query=
Finish
2856


,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet,PostDate
0,943747,https://lenta.ru/news/2020/01/01/org/,В России утвердили стандарты органической еды,1577838649,1577838649,1,1,0,0,4,"[7, 8]",https://icdn.lenta.ru/images/2019/12/31/13/201...,1577838649,Фото: Rupert Oberhäuser / Globallookpress.com ...,Вступил в силу принятый в 2018 году закон,Фото: Rupert Oberhäuser / ... для производител...,2020-01-01
1,943634,https://lenta.ru/news/2020/01/01/theaters/,Российские театры освободили от налогов,1577840882,1577840883,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2019/12/30/20/201...,1577840882,Фото: Константин Кокошкин / «Коммерсантъ» Росс...,Обнулить базу удастся при условии строго следо...,Фото: Константин Кокошкин / «Коммерсантъ»... п...,2020-01-01
2,943811,https://lenta.ru/news/2020/01/01/tariff/,Покупать в интернете за границей стало сложнее,1577847420,1577958238,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2019/12/31/17/201...,1577847420,Фото: Depositphotos Покупать в интернете за гр...,Порог беспошлинного ввоза снижен до 200 евро з...,"Фото: Depositphotos Покупать в интернете ..., ...",2020-01-01
3,943816,https://lenta.ru/news/2020/01/01/gas/,На Украине подорожал газ,1577850424,1577850425,1,1,0,0,4,"[7, 8, 198]",https://icdn.lenta.ru/images/2019/12/31/18/201...,1577850424,Фото: Sean Gallup / Getty Images Цена на газ д...,Стоимость тысячи кубометров для населения дост...,"Фото: Sean Gallup / Getty Images Цена на ...,3...",2020-01-01
4,943729,https://lenta.ru/news/2020/01/01/self/,В России стало больше самозанятых,1577854128,1577854128,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2019/12/31/11/201...,1577854128,Фото: Олег Харсеев / «Коммерсантъ» В России ст...,Режим распространили на 19 новых регионов,Фото: Олег Харсеев / «Коммерсантъ» В ... распр...,2020-01-01


# Loading of a prices

In [257]:
import yfinance as yf


tickers_list = ['YNDX.ME', 'GAZP.ME','SBER.ME']

# Import pandas
findata = pd.DataFrame(columns=tickers_list)

# Fetch the data

for ticker in tickers_list:
    loaded_data = yf.download(ticker, dateFrom, dateTo)
    findata[ticker] =loaded_data['Adj Close']

# Print first 5 rows of the data
findata.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,YNDX.ME,GAZP.ME,SBER.ME
Date,,,
2020-01-03,2685.000000,228.495285,181.040680
2020-01-06,2672.000000,226.333817,180.259720
2020-01-08,2677.000000,225.663345,183.987045
2020-01-09,2728.399902,224.145935,183.163467
2020-01-10,2725.000000,222.231506,183.305466


In [258]:
findata.head(5)

,YNDX.ME,GAZP.ME,SBER.ME
Date,,,
2020-01-03,2685.000000,228.495285,181.040680
2020-01-06,2672.000000,226.333817,180.259720
2020-01-08,2677.000000,225.663345,183.987045
2020-01-09,2728.399902,224.145935,183.163467
2020-01-10,2725.000000,222.231506,183.305466


# Очистка текстов и преобразование данных
## Lenta datasetss analysis

In [259]:

tbl.head(3)#, findata.head(3)


,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet,PostDate
0,943747,https://lenta.ru/news/2020/01/01/org/,В России утвердили стандарты органической еды,1577838649,1577838649,1,1,0,0,4,"[7, 8]",https://icdn.lenta.ru/images/2019/12/31/13/201...,1577838649,Фото: Rupert Oberhäuser / Globallookpress.com ...,Вступил в силу принятый в 2018 году закон,Фото: Rupert Oberhäuser / ... для производител...,2020-01-01
1,943634,https://lenta.ru/news/2020/01/01/theaters/,Российские театры освободили от налогов,1577840882,1577840883,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2019/12/30/20/201...,1577840882,Фото: Константин Кокошкин / «Коммерсантъ» Росс...,Обнулить базу удастся при условии строго следо...,Фото: Константин Кокошкин / «Коммерсантъ»... п...,2020-01-01
2,943811,https://lenta.ru/news/2020/01/01/tariff/,Покупать в интернете за границей стало сложнее,1577847420,1577958238,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2019/12/31/17/201...,1577847420,Фото: Depositphotos Покупать в интернете за гр...,Порог беспошлинного ввоза снижен до 200 евро з...,"Фото: Depositphotos Покупать в интернете ..., ...",2020-01-01


In [260]:
for idx, finrow in findata.iterrows():
    curdate = str(idx.date())
    ydx = finrow['YNDX.ME']

    tbl.loc[tbl['PostDate'] == curdate,'YNDX.ME']=ydx
    
tbl


,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet,PostDate,YNDX.ME
0,943747,https://lenta.ru/news/2020/01/01/org/,В России утвердили стандарты органической еды,1577838649,1577838649,1,1,0,0,4,"[7, 8]",https://icdn.lenta.ru/images/2019/12/31/13/201...,1577838649,Фото: Rupert Oberhäuser / Globallookpress.com ...,Вступил в силу принятый в 2018 году закон,Фото: Rupert Oberhäuser / ... для производител...,2020-01-01,NaN
1,943634,https://lenta.ru/news/2020/01/01/theaters/,Российские театры освободили от налогов,1577840882,1577840883,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2019/12/30/20/201...,1577840882,Фото: Константин Кокошкин / «Коммерсантъ» Росс...,Обнулить базу удастся при условии строго следо...,Фото: Константин Кокошкин / «Коммерсантъ»... п...,2020-01-01,NaN
2,943811,https://lenta.ru/news/2020/01/01/tariff/,Покупать в интернете за границей стало сложнее,1577847420,1577958238,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2019/12/31/17/201...,1577847420,Фото: Depositphotos Покупать в интернете за гр...,Порог беспошлинного ввоза снижен до 200 евро з...,"Фото: Depositphotos Покупать в интернете ..., ...",2020-01-01,NaN
3,943816,https://lenta.ru/news/2020/01/01/gas/,На Украине подорожал газ,1577850424,1577850425,1,1,0,0,4,"[7, 8, 198]",https://icdn.lenta.ru/images/2019/12/31/18/201...,1577850424,Фото: Sean Gallup / Getty Images Цена на газ д...,Стоимость тысячи кубометров для населения дост...,"Фото: Sean Gallup / Getty Images Цена на ...,3...",2020-01-01,NaN
4,943729,https://lenta.ru/news/2020/01/01/self/,В России стало больше самозанятых,1577854128,1577854128,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2019/12/31/11/201...,1577854128,Фото: Олег Харсеев / «Коммерсантъ» В России ст...,Режим распространили на 19 новых регионов,Фото: Олег Харсеев / «Коммерсантъ» В ... распр...,2020-01-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2851,973909,https://lenta.ru/news/2020/05/05/oil/,Цены на нефть снова выросли,1588704600,1588704753,1,1,0,0,4,[8],https://icdn.lenta.ru/images/2020/05/05/21/202...,1588704600,Фото: Eric Gay / AP Цены на нефть марки Brent ...,Стоимость июльских фьючерсов Brent достигла 31...,Фото: Eric Gay / AP Цены на нефть марки Brent ...,2020-05-04,2828.800049
2852,973918,https://lenta.ru/news/2020/05/05/9mayopen/,В Германии предложили открыть рестораны и кафе...,1588711380,1588712206,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2020/05/05/23/202...,1588711380,Фото: Игорь Зарембо / РИА Новости Министры эко...,Закрытые из-за эпидемии коронавируса заведения...,Фото: Игорь Зарембо / РИА Новости ... и кафе в...,2020-05-04,2828.800049
2853,973925,https://lenta.ru/news/2020/05/06/money/,Минфин оценил возможность раздачи россиянам де...,1588714560,1588715566,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2020/05/06/00/202...,1588714560,Фото: Globallookpress.com Глава Минфина Антон ...,"Антон Силуанов заявил, что Россия могла бы пой...",Фото: Globallookpress.com Глава Минфина ... ра...,2020-05-04,2828.800049
2854,973926,https://lenta.ru/news/2020/05/06/snake/,В России назвали ожидаемую стоимость нефти в 2...,1588715520,1588746410,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2020/05/06/00/202...,1588715520,Антон Силуанов Фото: Дмитрий Астахов / РИА Нов...,Средняя цена на нефть будет 30 долларов за бар...,Антон Силуанов Фото: Дмитрий Астахов / ... на ...,2020-05-04,2828.800049


In [261]:
import spacy
# nlp = spacy.load("ru_core_news_sm")
import ru_core_news_sm
nlp = ru_core_news_sm.load()

In [262]:
stopwords = nlp.Defaults.stop_words
print(f'Spacy english stopwords size: {len(stopwords)}', end='\n\n')
' '.join(stopwords)

Spacy english stopwords size: 768



'будешь нибудь поелику наподобие мной очень их без нечто одних той было ура некому своей нём но котором даром сначала та увы никого такую бац ведь паче моги хорошо вернее хоть больше вовсе назад и авось который моё чуть самими таков данные моего каждый ней хотел благодаря наипаче иль кто мочь нему яко тоже ку чём внизу моим ими находиться стал вполне любом рано такими ест вон иначе само похожем обычно тотчас взаправду вправду близко з собой эй очевидно навряд уж этой отсюда никем тот х можете неужели своя наперекор накануне особые которому с всём ти вас всеми твоя всякого еле самых непрерывно самой должны об том гораздо бывают которым наверняка твоих данному сие я насилу своими могите одно вся какие любыми необходимым мне мои другая э казался при этого любого то ага е самый зато вперекор данной посему любую может аж могло во ау эдакий ею вопреки нужного казалась которого некогда нынче довольно чем меня щ от лишь всею твоим зачем дальше данная которых над ем ел безусловно ваша сверх ща 

In [264]:
tbl['cleaned_text'] = tbl['rightcol'].apply(
    lambda x: ' '.join(
        token.lemma_.lower() for token in nlp(x) if 
        not token.is_stop 
        and not token.is_punct
        and not token.is_digit
        and not token.like_email
        and not token.like_num
        and not token.is_space
    )
)
tbl = tbl[tbl['YNDX.ME'].notna()]
tbl


,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet,PostDate,YNDX.ME,cleaned_text
19,944024,https://lenta.ru/news/2020/01/03/brent_up/,Цены на нефть взлетели после убийства главы ир...,1578024360,1578030941,1,1,0,0,4,[52],https://icdn.lenta.ru/images/2020/01/03/07/202...,1578024360,Фото: Pixabay Стоимость фьючерса нефти марки B...,Стоимость фьючерса Brent повысилась после смер...,Фото: Pixabay Стоимость фьючерса нефти ... рев...,2020-01-03,2685.000000,стоимость фьючерс brent повыситься смерть гене...
20,944034,https://lenta.ru/news/2020/01/03/prekratili/,Россия прекратила подачу нефти на белорусские ...,1578036180,1578040654,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2020/01/03/10/202...,1578036180,Фото: Евгений Самарин / РИА Новости Россия с 1...,Предприятия пока работают на оставшихся запасах,Фото: Евгений Самарин / РИА Новости ... нефтеп...,2020-01-03,2685.000000,предприятие работать остаться запас
21,944038,https://lenta.ru/news/2020/01/03/belneft/,В Белоруссии прокомментировали остановку поста...,1578038160,1578040788,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2020/01/03/10/202...,1578038160,Фото: Евгений Одиноков / РИА Новости В Белорус...,Минск занимается проработкой контрактов,Фото: Евгений Одиноков / РИА Новости В ... до ...,2020-01-03,2685.000000,минск заниматься проработка контракт
22,944079,https://lenta.ru/news/2020/01/03/gaz/,Названа цена на российский газ для Белоруссии,1578057060,1578057464,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2020/01/03/16/202...,1578057060,Фото: Егор Еремов / РИА Новости В январе-февра...,Минск будет получать топливо по цене в 127 дол...,Фото: Егор Еремов / РИА Новости В январе-... д...,2020-01-03,2685.000000,минск получать топливо цена доллар тысяча куби...
23,944090,https://lenta.ru/news/2020/01/03/no_oil_sorry/,Белоруссия остановила экспорт нефтепродуктов,1578063780,1578065925,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2020/01/03/17/202...,1578063780,Фото: Friso Gentsch / dpa / Global Look Press ...,Прекращение их вывоза за границу подтвердили в...,Фото: Friso Gentsch / dpa / Global Look ... ос...,2020-01-03,2685.000000,прекращение вывоз граница подтвердить белнефте...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2851,973909,https://lenta.ru/news/2020/05/05/oil/,Цены на нефть снова выросли,1588704600,1588704753,1,1,0,0,4,[8],https://icdn.lenta.ru/images/2020/05/05/21/202...,1588704600,Фото: Eric Gay / AP Цены на нефть марки Brent ...,Стоимость июльских фьючерсов Brent достигла 31...,Фото: Eric Gay / AP Цены на нефть марки Brent ...,2020-05-04,2828.800049,стоимость июльский фьючерс brent достигнуть до...
2852,973918,https://lenta.ru/news/2020/05/05/9mayopen/,В Германии предложили открыть рестораны и кафе...,1588711380,1588712206,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2020/05/05/23/202...,1588711380,Фото: Игорь Зарембо / РИА Новости Министры эко...,Закрытые из-за эпидемии коронавируса заведения...,Фото: Игорь Зарембо / РИА Новости ... и кафе в...,2020-05-04,2828.800049,закрыть эпидемия коронавирус заведение вероятн...
2853,973925,https://lenta.ru/news/2020/05/06/money/,Минфин оценил возможность раздачи россиянам де...,1588714560,1588715566,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2020/05/06/00/202...,1588714560,Фото: Globallookpress.com Глава Минфина Антон ...,"Антон Силуанов заявил, что Россия могла бы пой...",Фото: Globallookpress.com Глава Минфина ... ра...,2020-05-04,2828.800049,антон силуанов заявить россия пойти рубль миро...
2854,973926,https://lenta.ru/news/2020/05/06/snake/,В России назвали ожидаемую стоимость нефти в 2...,1588715520,1588746410,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2020/05/06/00/202...,1588715520,Антон Силуанов Фото: Дмитрий Астахов / РИА Нов...,Средняя цена на нефть будет 30 долларов за бар...,Антон Силуанов Фото: Дмитрий Астахов / ... на ...,2020-05-04,2828.800049,средний цена нефть доллар баррель


In [266]:
from sklearn.feature_extraction.text import TfidfTransformer,  TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(tbl['cleaned_text'], tbl['YNDX.ME'], random_state=2023)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X_train)
vectorizer.get_feature_names_out()[:100]

array(['00', '05', '08', '10', '12', '141', '15', '18', '19', '1980',
       '1990', '20', '2008', '2018', '2019', '2020', '2021', '21', '32',
       '35', '45', '50', '800', '92', 'a350', 'a380', 'adidas',
       'agroidea', 'airbus', 'alibaba', 'amazon', 'america', 'ap1',
       'apple', 'aramco', 'b2b', 'baer', 'bahn', 'bank', 'banking', 'bat',
       'beluga', 'billionaires', 'bioprocessing', 'bloomberg', 'boeing',
       'brand', 'brent', 'brexit', 'burger', 'cadillac', 'cdn',
       'chesapeake', 'chrysler', 'cif', 'citigroup', 'clever',
       'conference', 'consulting', 'coosur', 'covid', 'credit', 'cruiser',
       'cybertruck', 'danone', 'dax', 'deutsche', 'development', 'dns',
       'domino', 'en', 'energy', 'escalade', 'exxon', 'finance', 'fitch',
       'foods', 'forbes', 'forex', 'frank', 'futuretech', 'galactic',
       'gdr', 'general', 'global', 'globus', 'goldman', 'group',
       'guggenheim', 'gunvor', 'honda', 'hsbc', 'inc', 'index',
       'intagram', 'inteco', '

In [268]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_percentage_error

pipe = Pipeline(
    steps=[
        ('tfidf', TfidfVectorizer()),
        ('sgd_regr',SGDRegressor() )
        
    ]
).fit(X_train, y_train)
preds = pipe.predict(X_test)

mape = mean_absolute_percentage_error(y_test, preds)
mape


/Users/anastasiia/opt/miniconda3/envs/nlp_project/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


0.08920620385004589